In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import tensorflow
from tensorflow import keras
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import classification_report , confusion_matrix

In [80]:

xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [81]:
train_hist = xmr.history(period='10y')
test_hist = train_hist.copy()
train_hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-11-09 00:00:00+00:00,112.531998,123.404999,112.219002,120.779999,86864600,0.0,0.0
2017-11-10 00:00:00+00:00,121.344002,121.665001,101.757004,105.585999,84614000,0.0,0.0
2017-11-11 00:00:00+00:00,105.750000,127.106003,103.877998,119.615997,107708000,0.0,0.0
2017-11-12 00:00:00+00:00,119.597000,133.675003,110.617996,123.856003,144948000,0.0,0.0
2017-11-13 00:00:00+00:00,128.960007,136.528000,120.921997,123.402000,116200000,0.0,0.0


In [82]:
train_hist.iloc[2159]

Open            1.552195e+02
High            1.564820e+02
Low             1.537633e+02
Close           1.561912e+02
Volume          5.985898e+07
Dividends       0.000000e+00
Stock Splits    0.000000e+00
Name: 2023-10-08 00:00:00+00:00, dtype: float64

In [83]:
train_start = [2017,11,9]
train_stop = [2023,9,7]
test_start = [2023,9,8]
test_stop=[2023,10,7]

In [84]:
train_hist = train_hist.reset_index()
test_hist = test_hist.reset_index()

In [85]:
xmr_hash = pd.read_csv('XMR_hash.csv')
xmr_difficulty = pd.read_csv('XMR_difficulty.csv')

In [86]:
xmr_hash.head()

,Unnamed: 0,time_stamp,hash,time
0,0,1.442190e+12,12514082.15,2015-09-14T00:00:00Z
1,1,1.442280e+12,13361330.16,2015-09-15T00:00:00Z
2,2,1.442360e+12,13065948.13,2015-09-16T00:00:00Z
3,3,1.442450e+12,12663854.33,2015-09-17T00:00:00Z
4,4,1.442530e+12,12198340.44,2015-09-18T00:00:00Z


In [87]:
# change format to datetime
xmr_hash['time'] = pd.to_datetime(xmr_hash['time'])
xmr_difficulty['time'] = pd.to_datetime(xmr_difficulty['time'])
train_hist['Date'] = pd.to_datetime(train_hist['Date'])
test_hist['Date'] = pd.to_datetime(test_hist['Date'])

In [88]:
index_1 = train_hist.index[(train_hist['Date'].dt.year == train_start[0])
                    & (train_hist['Date'].dt.month == train_start[1])
                    & (train_hist['Date'].dt.day == train_start[2] )].tolist()[0]

index_2 = train_hist.index[(train_hist['Date'].dt.year == train_stop[0])
                    & (train_hist['Date'].dt.month == train_stop[1])
                    & (train_hist['Date'].dt.day == train_stop[2] )].tolist()[0]

train_hist = train_hist.iloc[index_1 : index_2+1]

In [89]:
index_1 = xmr_hash.index[(xmr_hash['time'].dt.year == train_start[0])
                    & (xmr_hash['time'].dt.month == train_start[1])
                    & (xmr_hash['time'].dt.day == train_start[2] )].tolist()[0]

index_2 = xmr_hash.index[(xmr_hash['time'].dt.year == train_stop[0])
                    & (xmr_hash['time'].dt.month == train_stop[1])
                    & (xmr_hash['time'].dt.day == train_stop[2] )].tolist()[0]

train_xmr_hash = xmr_hash.iloc[index_1 : index_2+1]

In [90]:
index_1 = xmr_difficulty.index[(xmr_difficulty['time'].dt.year == train_start[0])
                    & (xmr_difficulty['time'].dt.month == train_start[1])
                    & (xmr_difficulty['time'].dt.day == train_start[2] )].tolist()[0]

index_2 = xmr_difficulty.index[(xmr_difficulty['time'].dt.year == train_stop[0])
                    & (xmr_difficulty['time'].dt.month == train_stop[1])
                    & (xmr_difficulty['time'].dt.day == train_stop[2] )].tolist()[0]

train_xmr_difficulty = xmr_difficulty.iloc[index_1 : index_2+1]

In [91]:
train_hist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2017-11-09 00:00:00+00:00,112.531998,123.404999,112.219002,120.779999,86864600,0.0,0.0
1,2017-11-10 00:00:00+00:00,121.344002,121.665001,101.757004,105.585999,84614000,0.0,0.0
2,2017-11-11 00:00:00+00:00,105.750000,127.106003,103.877998,119.615997,107708000,0.0,0.0
3,2017-11-12 00:00:00+00:00,119.597000,133.675003,110.617996,123.856003,144948000,0.0,0.0
4,2017-11-13 00:00:00+00:00,128.960007,136.528000,120.921997,123.402000,116200000,0.0,0.0
...,...,...,...,...,...,...,...,...
2124,2023-09-03 00:00:00+00:00,140.088516,141.391968,139.297470,140.839981,49564266,0.0,0.0
2125,2023-09-04 00:00:00+00:00,140.842804,143.770203,140.776291,141.170868,63835398,0.0,0.0
2126,2023-09-05 00:00:00+00:00,141.176682,141.299179,137.673279,137.781372,54468002,0.0,0.0
2127,2023-09-06 00:00:00+00:00,137.780243,142.644989,137.635544,142.113235,67734044,0.0,0.0


In [92]:
index_1 = test_hist.index[(test_hist['Date'].dt.year == test_start[0])
                    & (test_hist['Date'].dt.month == test_start[1])
                    & (test_hist['Date'].dt.day == test_start[2] )].tolist()[0]

index_2 = test_hist.index[(test_hist['Date'].dt.year == test_stop[0])
                    & (test_hist['Date'].dt.month == test_stop[1])
                    & (test_hist['Date'].dt.day == test_stop[2] )].tolist()[0]

test_hist = test_hist.iloc[index_1 : index_2+1]

In [93]:
index_1 = xmr_hash.index[(xmr_hash['time'].dt.year == test_start[0])
                    & (xmr_hash['time'].dt.month == test_start[1])
                    & (xmr_hash['time'].dt.day == test_start[2] )].tolist()[0]

index_2 = xmr_hash.index[(xmr_hash['time'].dt.year == test_stop[0])
                    & (xmr_hash['time'].dt.month == test_stop[1])
                    & (xmr_hash['time'].dt.day == test_stop[2] )].tolist()[0]

test_xmr_hash = xmr_hash.iloc[index_1 : index_2+1]

In [94]:
index_1 = xmr_difficulty.index[(xmr_difficulty['time'].dt.year == test_start[0])
                    & (xmr_difficulty['time'].dt.month == test_start[1])
                    & (xmr_difficulty['time'].dt.day == test_start[2] )].tolist()[0]

index_2 = xmr_difficulty.index[(xmr_difficulty['time'].dt.year == test_stop[0])
                    & (xmr_difficulty['time'].dt.month == test_stop[1])
                    & (xmr_difficulty['time'].dt.day == test_stop[2] )].tolist()[0]

test_xmr_difficulty = xmr_difficulty.iloc[index_1 : index_2+1]

In [95]:
test_hist.shape

(30, 8)

In [96]:
train_hist = train_hist.rename(columns={'Date':'time'})
test_hist = test_hist.rename(columns={'Date':'time'})


In [97]:
test_xmr_hash.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)
train_xmr_hash.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)
test_xmr_difficulty.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)
train_xmr_difficulty.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)

C:\Users\iliya\AppData\Local\Temp\ipykernel_23460\3560016450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_xmr_hash.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)
C:\Users\iliya\AppData\Local\Temp\ipykernel_23460\3560016450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_xmr_hash.drop(['Unnamed: 0','time_stamp'],axis=1,inplace=True)
C:\Users\iliya\AppData\Local\Temp\ipykernel_23460\3560016450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [99]:
train_xmr_hash.head()

,hash,time
784,233898872.2,2017-11-09 00:00:00+00:00
785,233778227.0,2017-11-10 00:00:00+00:00
786,233778227.0,2017-11-11 00:00:00+00:00
787,233778227.0,2017-11-12 00:00:00+00:00
788,233778227.0,2017-11-13 00:00:00+00:00


In [104]:
test_xmr_difficulty.tail()

,difficulty,time
3590,3.669300e+11,2023-10-06 20:00:00+00:00
3591,3.692265e+11,2023-10-06 21:00:00+00:00
3592,3.715270e+11,2023-10-06 22:00:00+00:00
3593,3.728095e+11,2023-10-06 23:00:00+00:00
3594,3.710222e+11,2023-10-07 00:00:00+00:00


In [102]:
train_hist = train_hist.merge(xmr_hash, on='time', how='left')
test_hist = test_hist.merge(xmr_hash, on='time', how='left')

In [103]:
train_hist.head()

,time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Unnamed: 0,time_stamp,hash
0,2017-11-09 00:00:00+00:00,112.531998,123.404999,112.219002,120.779999,86864600,0.0,0.0,784.0,1.510190e+12,233898872.2
1,2017-11-10 00:00:00+00:00,121.344002,121.665001,101.757004,105.585999,84614000,0.0,0.0,785.0,1.510270e+12,233778227.0
2,2017-11-11 00:00:00+00:00,105.750000,127.106003,103.877998,119.615997,107708000,0.0,0.0,786.0,1.510360e+12,233778227.0
3,2017-11-12 00:00:00+00:00,119.597000,133.675003,110.617996,123.856003,144948000,0.0,0.0,787.0,1.510440e+12,233778227.0
4,2017-11-13 00:00:00+00:00,128.960007,136.528000,120.921997,123.402000,116200000,0.0,0.0,788.0,1.510530e+12,233778227.0


In [105]:
test_hist.tail()

,time,Open,High,Low,Close,Volume,Dividends,Stock Splits,Unnamed: 0,time_stamp,hash
25,2023-10-03 00:00:00+00:00,146.193024,147.805603,145.165909,147.151352,49389643,0.0,0.0,3495,1.696290e+12,2.998112e+09
26,2023-10-04 00:00:00+00:00,147.168442,150.702347,145.940781,150.469055,59400400,0.0,0.0,3519,1.696380e+12,2.732839e+09
27,2023-10-05 00:00:00+00:00,150.474197,151.328369,148.565491,149.623718,55704972,0.0,0.0,3543,1.696460e+12,2.838379e+09
28,2023-10-06 00:00:00+00:00,149.623337,152.669296,148.641647,151.992264,49535004,0.0,0.0,3567,1.696550e+12,2.831227e+09
29,2023-10-07 00:00:00+00:00,151.988235,155.247528,151.100983,155.212143,61159796,0.0,0.0,3591,1.696640e+12,3.091852e+09


In [106]:
train_hist.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Unnamed: 0,time_stamp,hash
count,2129.000000,2129.000000,2129.000000,2129.000000,2.129000e+03,2129.0,2129.0,2120.000000,2.120000e+03,2.120000e+03
mean,150.092462,155.524666,144.179966,150.072704,2.465300e+08,0.0,0.0,1843.500000,1.602479e+12,1.634458e+09
std,82.255117,86.759667,77.124310,82.200046,1.006991e+09,0.0,0.0,612.135606,5.292975e+10,1.009103e+09
min,33.084614,36.645374,26.703888,33.010323,1.016030e+07,0.0,0.0,784.000000,1.510190e+12,2.337782e+08
25%,82.517708,85.377037,79.266937,82.498695,6.533673e+07,0.0,0.0,1313.750000,1.556738e+12,5.053223e+08
50%,143.495956,146.876007,140.509567,143.487396,1.028917e+08,0.0,0.0,1843.500000,1.602505e+12,1.546591e+09
75%,193.847900,201.756378,186.152191,193.853897,1.911581e+08,0.0,0.0,2373.250000,1.648272e+12,2.622652e+09
max,483.709991,517.620239,453.390991,483.583618,2.895912e+10,0.0,0.0,2903.000000,1.694040e+12,3.462946e+09


In [107]:
test_hist.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Unnamed: 0,time_stamp,hash
count,30.000000,30.000000,30.000000,30.000000,3.000000e+01,30.0,30.0,30.000000,3.000000e+01,3.000000e+01
mean,145.617495,147.352445,144.219802,146.016092,4.639975e+07,0.0,0.0,3243.300000,1.695384e+12,2.457939e+09
std,2.736729,2.859642,2.769494,3.209925,1.005275e+07,0.0,0.0,210.776422,7.605651e+08,3.003972e+08
min,139.932358,142.853256,138.178223,139.930466,3.125585e+07,0.0,0.0,2904.000000,1.694130e+12,1.984907e+09
25%,143.775639,145.268864,142.312241,144.028107,3.879257e+07,0.0,0.0,3069.000000,1.694760e+12,2.210605e+09
50%,145.782822,147.265495,144.801239,145.929565,4.490281e+07,0.0,0.0,3243.000000,1.695385e+12,2.364028e+09
75%,147.086151,148.512032,145.665337,147.300880,5.537239e+07,0.0,0.0,3417.000000,1.696010e+12,2.723430e+09
max,151.988235,155.247528,151.100983,155.212143,6.811655e+07,0.0,0.0,3591.000000,1.696640e+12,3.091852e+09


In [109]:
train_hist.drop(['Dividends','Stock Splits','Unnamed: 0'],axis=1 , inplace=True)
test_hist.drop(['Dividends','Stock Splits','Unnamed: 0'],axis=1 , inplace=True)
train_hist.tail()

,time,Open,High,Low,Close,Volume,time_stamp,hash
2124,2023-09-03 00:00:00+00:00,140.088516,141.391968,139.297470,140.839981,49564266,1.693700e+12,2.164826e+09
2125,2023-09-04 00:00:00+00:00,140.842804,143.770203,140.776291,141.170868,63835398,1.693790e+12,2.323926e+09
2126,2023-09-05 00:00:00+00:00,141.176682,141.299179,137.673279,137.781372,54468002,1.693870e+12,2.157782e+09
2127,2023-09-06 00:00:00+00:00,137.780243,142.644989,137.635544,142.113235,67734044,1.693960e+12,2.158740e+09
2128,2023-09-07 00:00:00+00:00,142.107117,143.275955,141.168930,143.250748,62422431,1.694040e+12,2.015884e+09


In [110]:
test_hist.head()

,time,Open,High,Low,Close,Volume,time_stamp,hash
0,2023-09-08 00:00:00+00:00,143.257370,144.124344,140.928955,143.886154,46927009,1.694130e+12,1.984907e+09
1,2023-09-09 00:00:00+00:00,143.895950,144.001816,142.324234,143.406815,35896666,1.694220e+12,2.197674e+09
2,2023-09-10 00:00:00+00:00,143.409027,143.792282,141.827194,142.894440,33705148,1.694300e+12,2.105502e+09
3,2023-09-11 00:00:00+00:00,142.890076,143.619995,138.178223,139.930466,58031716,1.694390e+12,2.362412e+09
4,2023-09-12 00:00:00+00:00,139.932358,142.853256,139.286240,140.804672,57433672,1.694480e+12,2.131544e+09


In [111]:
label = list()
for i in range(train_hist.shape[0]-1):
    if train_hist['Close'].iloc[i]<=train_hist['Close'].iloc[i+1]:
        label.append(1)
    else :
        label.append(0)
        
label.append(0)

train_hist['label'] = np.array(label)


In [112]:
label = list()
for i in range(test_hist.shape[0]-1):
    if test_hist['Close'].iloc[i]<=test_hist['Close'].iloc[i+1]:
        label.append(1)
    else :
        label.append(0)

label.append(1)

test_hist['label'] = np.array(label)

In [113]:
test_hist.head()

,time,Open,High,Low,Close,Volume,time_stamp,hash,label
0,2023-09-08 00:00:00+00:00,143.257370,144.124344,140.928955,143.886154,46927009,1.694130e+12,1.984907e+09,0
1,2023-09-09 00:00:00+00:00,143.895950,144.001816,142.324234,143.406815,35896666,1.694220e+12,2.197674e+09,0
2,2023-09-10 00:00:00+00:00,143.409027,143.792282,141.827194,142.894440,33705148,1.694300e+12,2.105502e+09,0
3,2023-09-11 00:00:00+00:00,142.890076,143.619995,138.178223,139.930466,58031716,1.694390e+12,2.362412e+09,1
4,2023-09-12 00:00:00+00:00,139.932358,142.853256,139.286240,140.804672,57433672,1.694480e+12,2.131544e+09,1


In [115]:
test_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        30 non-null     datetime64[ns, UTC]
 1   Open        30 non-null     float64            
 2   High        30 non-null     float64            
 3   Low         30 non-null     float64            
 4   Close       30 non-null     float64            
 5   Volume      30 non-null     int64              
 6   time_stamp  30 non-null     float64            
 7   hash        30 non-null     float64            
 8   label       30 non-null     int32              
dtypes: datetime64[ns, UTC](1), float64(6), int32(1), int64(1)
memory usage: 2.1 KB


In [116]:
train_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        2129 non-null   datetime64[ns, UTC]
 1   Open        2129 non-null   float64            
 2   High        2129 non-null   float64            
 3   Low         2129 non-null   float64            
 4   Close       2129 non-null   float64            
 5   Volume      2129 non-null   int64              
 6   time_stamp  2120 non-null   float64            
 7   hash        2120 non-null   float64            
 8   label       2129 non-null   int32              
dtypes: datetime64[ns, UTC](1), float64(6), int32(1), int64(1)
memory usage: 141.5 KB
